<a href="https://colab.research.google.com/github/ahmedashrafomar/Bachelor/blob/main/Fine_tune_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi


Fri Nov 28 18:05:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import zipfile, os

zip_path = "/content/drive/MyDrive/Copy of FNNPK.zip"  # ✅ real file

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall("/content")

DATASET_PATH = "/content/FNNPK"  # folder created after unzip
print("✅ Dataset Path:", DATASET_PATH)
print("📁 Class folders:", os.listdir(DATASET_PATH))


✅ Dataset Path: /content/FNNPK
📁 Class folders: ['-N', '-K', '-P', 'FN']


In [8]:
!pip install -U -q "pillow<12.0,>=8.0"
!pip install -U -q git+https://github.com/huggingface/trl.git
!pip install -U -q transformers accelerate peft bitsandbytes qwen-vl-utils
!pip install -U -q datasets torchvision scikit-learn


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 134.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 59.8 MB/s et

In [9]:
import os

DATASET_PATH = "/content/FNNPK"  # ✅ change only if folder name differs

print("✅ Dataset Path:", DATASET_PATH)
print("📁 Classes:", os.listdir(DATASET_PATH))


✅ Dataset Path: /content/FNNPK
📁 Classes: ['-N', '-K', '-P', 'FN']


In [13]:
class_counts = {
    cls: len(os.listdir(os.path.join(DATASET_PATH, cls)))
    for cls in os.listdir(DATASET_PATH)
    if os.path.isdir(os.path.join(DATASET_PATH, cls))
}
print("\n📊 Original Class Distribution:", class_counts)



📊 Original Class Distribution: {'-N': 58, '-K': 72, '-P': 66, 'FN': 12}


In [14]:
import random
from PIL import Image
from torchvision import transforms
from collections import defaultdict

TARGET_COUNT = max(class_counts.values())
print("🎯 Target per class:", TARGET_COUNT)

augment = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(25),
    transforms.ColorJitter(0.3, 0.3, 0.3, 0.1),
    transforms.RandomResizedCrop(384, scale=(0.8, 1.0)),
])

all_entries = []
class_entries = defaultdict(list)

for cls in os.listdir(DATASET_PATH):
    path = os.path.join(DATASET_PATH, cls)
    if not os.path.isdir(path):
        continue

    imgs = [i for i in os.listdir(path) if i.lower().endswith(("jpg", "jpeg", "png"))]

    for img in imgs:
        class_entries[cls].append({
            "image_path": os.path.join(path, img),
            "label": cls,
            "is_augmented": False
        })

for cls, entries in class_entries.items():
    count = len(entries)
    if count < TARGET_COUNT:
        needed = TARGET_COUNT - count
        print(f"🌱 Augmenting '{cls}' → {count} ➜ {TARGET_COUNT} (+{needed})")
        originals = entries.copy()
        for _ in range(needed):
            original = random.choice(originals)
            class_entries[cls].append({
                "image_path": original["image_path"],
                "label": cls,
                "is_augmented": True
            })

for cls, entries in class_entries.items():
    all_entries.extend(entries)

random.shuffle(all_entries)

print("\n✅ Balanced Distribution:", {cls: len(e) for cls, e in class_entries.items()})
print("✅ Total Samples:", len(all_entries))

🎯 Target per class: 72
🌱 Augmenting '-N' → 58 ➜ 72 (+14)
🌱 Augmenting '-P' → 66 ➜ 72 (+6)
🌱 Augmenting 'FN' → 12 ➜ 72 (+60)

✅ Balanced Distribution: {'-N': 72, '-K': 72, '-P': 72, 'FN': 72}
✅ Total Samples: 288


In [15]:
from sklearn.model_selection import train_test_split

labels = [e["label"] for e in all_entries]

train_entries, val_entries = train_test_split(
    all_entries,
    test_size=0.15,
    stratify=labels,
    random_state=42
)

print("✅ Train Samples:", len(train_entries))
print("✅ Val Samples:", len(val_entries))

✅ Train Samples: 244
✅ Val Samples: 44


In [20]:
import torch
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor, BitsAndBytesConfig

model_id = "Qwen/Qwen2-VL-2B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16
)

processor = Qwen2VLProcessor.from_pretrained(model_id)

# ✅ FIX: Limit image resolution to prevent token mismatch
processor.image_processor.min_pixels = 256 * 28 * 28
processor.image_processor.max_pixels = 512 * 28 * 28
processor.tokenizer.model_max_length = 8192

print("✅ Model loaded with 4-bit quantization")

`torch_dtype` is deprecated! Use `dtype` instead!
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

✅ Model loaded with 4-bit quantization


In [16]:
# ============================================
# CELL 7: FORMAT DATASET
# ============================================
system_msg = "You are an expert plant pathologist. Identify the plant disease. Respond with ONLY the disease name."

instructions = [
    "What disease is affecting this plant?",
    "Identify the plant disease shown.",
    "Classify this plant disease.",
    "What problem does this plant leaf have?",
    "Name the visible disease."
]

def format_sample(entry, apply_aug=False):
    img = Image.open(entry["image_path"]).convert("RGB")
    img.thumbnail((512, 512), Image.Resampling.LANCZOS)
    if entry["is_augmented"] and apply_aug:
        img = augment(img)
    label = entry["label"].replace("_", " ")
    return {
        "images": [img],
        "messages": [
            {"role": "system", "content": [{"type": "text", "text": system_msg}]},
            {"role": "user", "content": [
                {"type": "image", "image": img},
                {"type": "text", "text": random.choice(instructions)}
            ]},
            {"role": "assistant", "content": [{"type": "text", "text": label}]}
        ]
    }

print("⏳ Formatting dataset...")
train_data = [format_sample(e, True) for e in train_entries]
val_data = [format_sample(e, False) for e in val_entries]
print(f"✅ Formatted {len(train_data)} train, {len(val_data)} val samples")

⏳ Formatting dataset...
✅ Formatted 244 train, 44 val samples


In [21]:
# ============================================
# CELL 8: CONFIGURE LoRA
# ============================================
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM"
)
print("✅ LoRA config ready")

✅ LoRA config ready


In [22]:
from transformers.processing_utils import ProcessorMixin

# ✅ Disable strict mismatch checking
ProcessorMixin._check_special_mm_tokens = lambda *args, **kwargs: None


In [23]:
# ============================================
# CELL 9: TRAIN MODEL
# ============================================
from trl import SFTConfig, SFTTrainer

training_args = SFTConfig(
    output_dir="qwen2vl-plant-disease",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    optim="adamw_torch_fused",
    learning_rate=1e-4,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    max_grad_norm=0.3,
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=30,
    save_strategy="steps",
    save_steps=60,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    bf16=True,
    remove_unused_columns=False,
    dataset_text_field=None,
    packing=False,
    push_to_hub=False,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    peft_config=lora_config,
    processing_class=processor,
)

print("🚀 Training started — expect ~1 hour on T4...")
trainer.train()
trainer.save_model("qwen2vl-plant-disease-final")
print("\n✅ Training complete — Model saved!")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


🚀 Training started — expect ~1 hour on T4...


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
30,11.296500,9.520535,8.314037,84127.000000,0.054979
60,7.294400,7.223382,7.490591,166573.000000,0.100206
90,6.953100,6.979466,7.190174,249083.000000,0.106752
120,6.903800,6.937172,7.135497,331881.000000,0.110400
150,6.906000,6.931934,7.123083,414736.000000,0.109472
180,6.876500,6.928037,7.119394,496982.000000,0.111077
210,6.876800,6.926922,7.117540,579497.000000,0.110768
240,6.893200,6.926180,7.115519,662472.000000,0.110769
270,6.888400,6.925732,7.114479,744864.000000,0.110768
300,6.893100,6.925862,7.114258,827446.000000,0.111077



✅ Training complete — Model saved!


In [ ]:
# ✅ Option 1: Upload file
from google.colab import files
print("\n📸 Upload a plant image to test the model")
test_upload = files.upload()
test_img_path = list(test_upload.keys())[0]

# ✅ Option 2: Or use existing file path (uncomment and modify)
# test_img_path = "/content/FNNPK/YOUR_CLASS_NAME/some_image.jpg"

test_img = Image.open(test_img_path).convert("RGB")
test_img.thumbnail((512, 512), Image.Resampling.LANCZOS)

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "What disease is affecting this plant?"}
    ]}
]

prompt = processor.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

inputs = processor(
    text=[prompt],
    images=[test_img],
    return_tensors="pt",
    padding=True
).to(model.device)

model.eval()

with torch.no_grad():
    with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        output_ids = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=False
        )

prediction = processor.batch_decode(
    output_ids[:, inputs.input_ids.shape[1]:],
    skip_special_tokens=True
)[0]

print("\n" + "="*50)
print(f"✅ PREDICTION: {prediction}")
print("="*50)


📸 Upload a plant image to test the model


Saving k_4.png to k_4 (2).png

✅ PREDICTION: -N


In [ ]:
# Test on a known "K" image from training data
import os

# Find a K image
k_folder = os.path.join(DATASET_PATH, "-K")  # Adjust if class name is different
k_images = [f for f in os.listdir(k_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]
test_img_path = os.path.join(k_folder, k_images[0])

print(f"Testing on: {test_img_path}")

test_img = Image.open(test_img_path).convert("RGB")
test_img.thumbnail((512, 512), Image.Resampling.LANCZOS)

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "What disease is affecting this plant?"}
    ]}
]

prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = processor(text=[prompt], images=[test_img], return_tensors="pt", padding=True).to(model.device)

model.eval()
with torch.no_grad():
    with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        output_ids = model.generate(**inputs, max_new_tokens=50, do_sample=False)

prediction = processor.batch_decode(output_ids[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]
print(f"\n✅ PREDICTION: {prediction}")
print(f"✅ EXPECTED: K")

Testing on: /content/FNNPK/-K/k_56.png

✅ PREDICTION: -K
✅ EXPECTED: K


In [ ]:
# Test one image from each class
for cls in os.listdir(DATASET_PATH):
    cls_path = os.path.join(DATASET_PATH, cls)
    if not os.path.isdir(cls_path):
        continue

    imgs = [f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    if not imgs:
        continue

    test_img = Image.open(os.path.join(cls_path, imgs[0])).convert("RGB")
    test_img.thumbnail((512, 512), Image.Resampling.LANCZOS)

    prompt = processor.apply_chat_template(
        [{"role": "system", "content": system_msg},
         {"role": "user", "content": [{"type": "image"}, {"type": "text", "text": "What disease is affecting this plant?"}]}],
        tokenize=False, add_generation_prompt=True
    )
    inputs = processor(text=[prompt], images=[test_img], return_tensors="pt", padding=True).to(model.device)

    with torch.no_grad():
        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            out = model.generate(**inputs, max_new_tokens=50, do_sample=False)

    pred = processor.batch_decode(out[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]
    print(f"Class: {cls:10} → Predicted: {pred}")

Class: -N         → Predicted: -N
Class: -K         → Predicted: -K
Class: -P         → Predicted: -P
Class: FN         → Predicted: -N


In [ ]:
!zip -r qwen2vl-plant-disease-final.zip qwen2vl-plant-disease-final
files.download("qwen2vl-plant-disease-final.zip")

  adding: qwen2vl-plant-disease-final/ (stored 0%)
  adding: qwen2vl-plant-disease-final/tokenizer_config.json (deflated 86%)
  adding: qwen2vl-plant-disease-final/training_args.bin (deflated 53%)
  adding: qwen2vl-plant-disease-final/chat_template.jinja (deflated 65%)
  adding: qwen2vl-plant-disease-final/video_preprocessor_config.json (deflated 55%)
  adding: qwen2vl-plant-disease-final/vocab.json (deflated 61%)
  adding: qwen2vl-plant-disease-final/preprocessor_config.json (deflated 52%)
  adding: qwen2vl-plant-disease-final/tokenizer.json (deflated 81%)
  adding: qwen2vl-plant-disease-final/merges.txt (deflated 57%)
  adding: qwen2vl-plant-disease-final/adapter_config.json (deflated 57%)
  adding: qwen2vl-plant-disease-final/special_tokens_map.json (deflated 69%)
  adding: qwen2vl-plant-disease-final/adapter_model.safetensors (deflated 8%)
  adding: qwen2vl-plant-disease-final/added_tokens.json (deflated 64%)
  adding: qwen2vl-plant-disease-final/README.md (deflated 65%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>